In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import cv2
from skimage.transform import rescale, resize
from skimage import color
import os

%load_ext autoreload
%autoreload 2
%matplotlib inline

import scipy.io
import os
import cv2
from PIL import Image
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # Change GPU here

import recon.load_scripts_extended as loading
from recon.helper_fns_extended import *

# Choose CPU or GPU here:
%run recon/fista_spectral_cupy.py CPU #GPU

if device == 'GPU':
    import cupy as np
    np.cuda.Device(0).use()
else:
    import numpy as np

from extended_fov_helpers import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
device =  CPU , using CPU and numpy


full flow would look like:
- generating a PSF
- using the PSF name, run through forward model
- once forward model is done, reconstruct
- quantify extent

## Generating PSFs

In [ ]:
## Random seed so we can reproduce the same results
SEED = 11
shape = (500, 500)
random.seed(SEED)
psf_index = 0

interact_manual(plot_rml_irs, truncate=widgets.FloatSlider(min=0, max=15, step=0.1, value=3, continuous_update=False),
         num_points=widgets.IntSlider(min=1, max=30, step=1, value=10, continuous_update=False),
         r=widgets.IntSlider(min=1, max=10, step=1, value=1, continuous_update=False),
         extent_h=widgets.IntSlider(min=0, max=shape[1], step=1, value=shape[1], continuous_update=False),
         extent_v=widgets.IntSlider(min=0, max=shape[0], step=1, value=shape[0], continuous_update=False),
         shape_h = shape[1], 
         shape_v = shape[0],
         pause=widgets.Checkbox(value=False, description='Pause', disabled=False, indent=False),
         save_png=widgets.Checkbox(value=False, description='Save as .png', disabled=False, indent=False),
         save_tiff=widgets.Checkbox(value=False, description='Save as .tiff', disabled=False, indent=False),
         save_npy=widgets.Checkbox(value=False, description='Save as .npy', disabled=False, indent=False))

In [ ]:
## Generating lots of PSFs
# for points in range(5, point_range, 5):
#     final_im = np.zeros(shape, np.uint8)
#     final_im, name = generate_rml_irs(final_im, bounds, truncate, points, rad, *extent)
#     if save:
#         save_final_im(final_im, name)

## Foward Model

In [6]:
GT_PATH = './penguin.PNG'
PSF_PATH = './3-6-24_rml_mono8_3000.tiff'
img_name = 'penguin'

In [ ]:
img, psf = load(GT_PATH, psf=PSF_PATH)
print("Original Image Dim:", img.shape)
img, scale_factor = rescale_img(img, 2, psf, gray=False, channel_axis=2)

# crop to the same aspect ratio
# scale_factor = (psf.shape[0] * 2, psf.shape[1]*2)
# img = center_crop(img, dim=scale_factor)
# print("X, Y scale factor: ", img.shape[:2])

plt.figure()
plt.imshow(img, cmap='gray')
plt.title("Input Image, 2x Sensor Height")

fwd, result_name, fname = forward(img, psf, gray=True, name=img_name)
plt.figure()
plt.imshow(fwd, cmap='gray')
plt.title(f"Sensor Measurement {fwd.shape[:2]}")

## Reconstruction Flow

In [ ]:
## image parameters
gray_image = False
f = 8

path = './'
results_path = 'results/'

psf_name = ''
object_name = 'patrick'

# PSF_PATH is rolled over from the forward model. 
# result_name is the save destination for the forward model
psf_path = PSF_PATH
img_path = path + result_name

psf, img, mask = preprocess(psf_path, img_path, f, gray_image=gray_image)

In [ ]:
## Recon Parameters
fista = fista_spectral_numpy(psf[:,:,1:2], mask[:,:,1:2], gray=gray_image) # green channel only

# set FISTA parameters
fista.iters = 50
# Default: tv, Options: 'native' for native sparsity, 'non-neg' for enforcing non-negativity only
fista.prox_method = 'tv'  
fista.tv_lambda  = 1e-2  #1e-3, 1e-2, 1e-1
fista.tv_lambdaw = 0.01 
fista.print_every = 20

## Run FISTA
out_img = fista.run(img)

## Display Results
# green channel only of PSF
cropped = out_img[0][0]
uncropped = out_img[0][1]

cropped_name = f'cropped_{psf_name}_{object_name}'
uncropped_name = f'uncropped_{psf_name}_{object_name}'

fig = plt.figure()
plt.imshow(preplot(cropped))
plt.axis('off') 
plt.imsave(path + cropped_name + '.png', preplot(cropped))


plt.figure()
plt.imshow(preplot(uncropped))
plt.axis('off') 
plt.imsave(path + uncropped_name + '.png', preplot(uncropped))

np.save(results_path + cropped_name + '.npy', cropped)
np.save(results_path + uncropped_name + '.npy', uncropped)

## Extent Quantification

In [ ]:
cropped_path = 'extended_fov/psf-generation-tests/penguin/cropped-measurenments.npy'
uncropped_path = 'extended_fov/psf-generation-tests/penguin/uncropped-measurenments.npy'

params = {
    'sigma': 0.5, 
    'low_threshold':0.2, 
    'high_threshold':1.0, 
    'show':True,
}

edges, h_increase, v_increase = get_extent_increase(cropped_path, uncropped_path, params, actual_shape=(744, 480))